In [61]:
from helpers import *
from rascal.representations import SphericalCovariants 
from rascal.neighbourlist.structure_manager import mask_center_atoms_by_species #change later
from rascal.representations import SphericalExpansion
from rascal.utils import ClebschGordanReal, compute_lambda_soap, spherical_expansion_reshape, xyz_to_spherical
from rascal.representations import SphericalInvariants
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

In [62]:
PATH_TRAIN = "./train_tensor/CSD-3k+S546_shift_tensors.xyz"
PATH_TEST = "./test_tensor/CSD-500+104-7_shift_tensors.xyz"

structures_train, tensors_train = load_CSD_data(PATH_TRAIN,prop_string="cs_tensor",random_subsample=100)
structures_test, tensors_test = load_CSD_data(PATH_TEST,prop_string="cs_tensor",random_subsample=100)#,random_subsample=20)


#-----mask----
for structure in structures_train: mask_center_atoms_by_species(structure,species_select=[6])
for structure in structures_test: mask_center_atoms_by_species(structure,species_select=[6])

#-----select tensors-----
train_tensors = np.array([tensor for structure in structures_train for tensor in structure.arrays["cs_tensor"][structure.arrays["center_atoms_mask"]]])
test_tensors = np.array([tensor for structure in structures_test for tensor in structure.arrays["cs_tensor"][structure.arrays["center_atoms_mask"]]])

#------transform tensors
cg = ClebschGordanReal(lmax=2)
tensors_train_coupled = cg.couple(xyz_to_spherical(train_tensors.reshape(train_tensors.shape[0],3,3)))
train_tensors_trans = tensors_train_coupled[(1,1)][2]

tensors_test_coupled = cg.couple(xyz_to_spherical(test_tensors.reshape(test_tensors.shape[0],3,3)))
test_tensors_trans = tensors_test_coupled[(1,1)][2]

In [63]:
train_tensors_trans.shape

(3296, 5)

In [64]:
HYPERS = {
    'soap_type': 'LambdaSpectrum',
    'interaction_cutoff': 3.0,
    'max_radial': 9,
    'max_angular': 9,
    'gaussian_sigma_constant': 0.5,
    'gaussian_sigma_type': 'Constant',
    'cutoff_smooth_width': 0.5,
    'radial_basis': 'GTO',
    'inversion_symmetry': True,
    'covariant_lambda' : 2,
    "cutoff_function_parameters":dict(rate=1,scale=3.5,exponent=4)
}

lsoap = SphericalCovariants(**HYPERS)
lX_train = lsoap.transform(structures_train).get_features(lsoap)
lX_test = lsoap.transform(structures_test).get_features(lsoap)

In [ ]:
hypers_2 = {
    'interaction_cutoff': 6,
    'max_radial': 9,
    'max_angular': 9,
    'gaussian_sigma_type': 'Constant',
    'gaussian_sigma_constant': 0.2,
    'cutoff_smooth_width': 0.3,
    'radial_basis': 'DVR'
}
hypers_2 = get_optimal_radial_basis_hypers(hypers_2, frames, expanded_max_radial=50)
CG = ClebschGordanReal(lmax=hypers_2["max_angular"])
spex = SphericalExpansion(**hypers_2)
feats = spex.transform(frames).get_features(spex)
ref_feats = 1e3*spherical_expansion_reshape(feats, **hypers_2)

In [65]:
ntot = len(lX_train)

In [66]:
len(np.arange(ntot)[::3])

1099

In [67]:
ntot

3296

In [68]:
ntot = len(lX_train)
itrain = np.arange(ntot)
np.random.seed(42)
np.random.shuffle(itrain)
isparse = np.arange(ntot)[::7]
np.random.shuffle(isparse)

In [69]:
lX

NameError: name 'lX' is not defined

In [70]:
def f2v(vec):
    t = ()
    for s in vec.shape:
        t += (s//5,5)
    return vec.reshape(t)

def v2f(vec):
    t = ()
    for s in vec.shape[:-1:2]:        
        t += tuple([s*5])
    if len(vec.shape)%2==1:
        t += tuple([-1])
    return vec.reshape(t)

In [78]:
lX_test  = np.moveaxis(lX_test.reshape((lX_test.shape[0],-1,5)),2,1)
lX_train = np.moveaxis(lX_train.reshape((lX_train.shape[0],-1,5)),2,1) #why this moveaxis??

In [90]:
lX_test = np.moveaxis(lX_test,1,2)

In [91]:
lX_test.shape

(5308, 30375, 5)

In [92]:
v2f(lX_test).shape

(26540, 30375)

In [93]:
np.moveaxis(lX_test, 2, 1).reshape((-1, lX_test.shape[1])).shape #does the same as v2f?


(26540, 30375)

In [94]:
lX_test.shape

(5308, 30375, 5)

In [87]:
test_tensors_trans.shape

(5308, 5)

In [86]:
test_tensors_trans.flatten().shape

(26540,)

In [72]:
lX_train.shape

(3296, 5, 30375)

In [73]:
lKNM = f2v(v2f(lX_train[itrain]) @ v2f(lX_train[isparse]).T)
lKMM = f2v(v2f(lX_train[isparse]) @ v2f(lX_train[isparse]).T)
lKM = f2v(v2f(lX_test) @ v2f(lX_train[isparse]).T)

In [97]:
len(isparse)

471

In [98]:
lKMM.shape

(471, 5, 471, 5)

In [74]:
wlsparse = f2v( np.linalg.lstsq(v2f(lKNM).T@v2f(lKNM) + 1e-8* v2f(lKMM), 
                           v2f(lKNM).T@v2f(train_tensors_trans[itrain]), rcond=None)[0] )

In [75]:
ylpred_sparse = f2v(v2f(lKM) @ v2f(wlsparse) )

In [76]:
ylpred_sparse

array([[-59.65362124,  10.9974187 , -12.13477327,  86.37655682,
         38.81831115],
       [-59.53716592, -10.89499325, -12.41964435, -86.32156216,
         38.90290481],
       [ 59.65258334,  10.99573367, -12.14126189, -86.37346957,
         38.81656139],
       ...,
       [  2.2830482 , -20.41357875,  20.49128175, -38.74074996,
          4.31568436],
       [ -2.27731818,  20.41229305,  20.48412093, -38.74640131,
          4.31017081],
       [ -2.27855596, -20.41219771,  20.48407182,  38.74530479,
          4.31134247]])

In [77]:
test_tensors_trans

array([[-39.09430759,  11.37381258,   2.69039706,  61.02126461,
        -65.85278382],
       [-38.98541314, -11.38144933,   2.64201964, -61.08242934,
        -65.89061403],
       [ 39.09416616,  11.37374186,   2.69056036, -61.02112319,
        -65.8526424 ],
       ...,
       [-11.06975666,  24.26988463,  22.88488864, -47.51948488,
         24.99558832],
       [ 11.06975666, -24.26988463,  22.88488864, -47.51948488,
         24.99558832],
       [ 11.06975666,  24.26988463,  22.88488864,  47.51955559,
         24.99572974]])